# Strands Agents with AgentCore Memory（短期記憶）


## はじめに

このチュートリアルでは、AgentCore **短期記憶**（生のイベント）を持つStrandsエージェントを使用して**パーソナルエージェント**を構築する方法を説明します。エージェントは`get_last_k_turns`を使用してセッション内の最近の会話を記憶し、ユーザーが戻ってきた時に会話をシームレスに継続できます。


### チュートリアルの詳細

| 項目                | 詳細                                                                             |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話型                                                                       |
| エージェントタイプ   | パーソナルエージェント                                                           |
| エージェントフレームワーク | Strands Agents                                                                   |
| LLMモデル           | Anthropic Claude Sonnet 3.7                                                      |
| チュートリアル構成要素 | AgentCore短期メモリ、AgentInitializedEventとMessageAddedEventフック               |
| 例の複雑度          | 初級                                                                             |

学習内容：
- 会話継続のための短期記憶の使用
- 最後のK会話ターンの取得
- リアルタイム情報のためのウェブ検索ツール
- 会話履歴によるエージェントの初期化

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10+
- AgentCoreメモリ権限を持つAWS認証情報
- AgentCoreメモリロールARN
- Amazon Bedrockモデルへのアクセス

環境をセットアップして始めましょう！

## ステップ1: セットアップとインポート

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
from datetime import datetime

# セットアップ
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("personal-agent")

In [ ]:
# インポート
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient

# 設定
REGION = os.getenv('AWS_REGION', 'us-west-2') # エージェント用のAWSリージョン
ACTOR_ID = "user_123" # 任意の一意の識別子（AgentID、User IDなど）
SESSION_ID = "personal_session_001" # 一意のセッション識別子

## ステップ2: ウェブ検索ツール

まず、エージェント用のシンプルなウェブ検索ツールを作成しましょう。

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """更新された情報をウェブで検索します。
    
    Args:
        keywords (str): 検索クエリのキーワード。
        region (str): 検索リージョン: wt-wt, us-en, uk-en, ru-ru, など。
        max_results (int | None): 返す結果の最大数。
    Returns:
        検索結果の辞書のリスト。
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "結果が見つかりませんでした。"
    except RatelimitException:
        return "レート制限に達しました。後でもう一度お試しください。"
    except DDGSException as e:
        return f"検索エラー: {e}"
    except Exception as e:
        return f"検索エラー: {str(e)}"

logger.info("✅ ウェブ検索ツールが準備できました")

## ステップ3: メモリリソースの作成
短期記憶の場合、ストラテジーを使用せずにメモリリソースを作成します。これにより、`get_last_k_turns`で取得できる生の会話ターンが保存されます。

In [ ]:
from botocore.exceptions import ClientError

# メモリクライアントを初期化
client = MemoryClient(region_name=REGION)
memory_name = "PersonalAgentMemory"

try:
    # ストラテジーなしでメモリリソースを作成（そのため短期記憶のみアクセス可能）
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # 短期記憶にはストラテジーなし
        description="パーソナルエージェント用の短期記憶",
        event_expiry_days=7, # 短期記憶の保持期間。これは最大365日まで可能。
    )
    memory_id = memory['id']
    logger.info(f"✅ メモリを作成しました: {memory_id}")
except ClientError as e:
    logger.info(f"❌ エラー: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # メモリが既に存在する場合、IDを取得
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"メモリが既に存在します。既存のメモリIDを使用: {memory_id}")
except Exception as e:
    # メモリ作成中のエラーを表示
    logger.error(f"❌ エラー: {e}")
    import traceback
    traceback.print_exc()
    # エラー時のクリーンアップ - 部分的に作成されたメモリを削除
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"メモリをクリーンアップしました: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"メモリのクリーンアップに失敗: {cleanup_error}")

## ステップ4: メモリフック

このステップでは、メモリ操作を自動化するカスタムの`MemoryHookProvider`クラスを定義します。フックとは、エージェントの実行ライフサイクルの特定の時点で実行される特別な関数です。作成しているメモリフックは2つの主要な機能を持ちます：
1. **最近の会話を読み込む**: `AgentInitializedEvent`フックを使用して、エージェントが初期化された時に自動的に最近の会話履歴を読み込みます。
2. **最後のメッセージを保存する**: 新しい会話メッセージを保存します。

これにより、手動管理なしにシームレスなメモリ体験を作成します。

In [ ]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str, actor_id: str, session_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
        self.actor_id = actor_id
        self.session_id = session_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """エージェント開始時に最近の会話履歴を読み込み"""
        try:
            # メモリから最後の5会話ターンを読み込み
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                k=5
            )
            
            if recent_turns:
                # コンテキスト用に会話履歴をフォーマット
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # エージェントのシステムプロンプトにコンテキストを追加
                event.agent.system_prompt += f"\n\n最近の会話:\n{context}"
                logger.info(f"✅ {len(recent_turns)}個の会話ターンを読み込みました")
                
        except Exception as e:
            logger.error(f"メモリ読み込みエラー: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """メッセージをメモリに保存"""
        messages = event.agent.messages
        try:
            self.memory_client.create_event(
                memory_id=self.memory_id,
                actor_id=self.actor_id,
                session_id=self.session_id,
                messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
            )
        except Exception as e:
            logger.error(f"メモリ保存エラー: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # メモリフックを登録
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## ステップ5: ウェブ検索機能付きパーソナルエージェントの作成

In [ ]:
def create_personal_agent():
    """メモリとウェブ検索機能付きパーソナルエージェントを作成"""
    agent = Agent(
        name="PersonalAssistant",
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # または希望のモデル
        system_prompt=f"""あなたはウェブ検索機能付きの役立つパーソナルアシスタントです。
        
        あなたができること:
        - 一般的な質問と情報検索
        - 現在の情報のためのウェブ検索
        - 個人的なタスク管理
        
        現在の情報が必要な場合は、websearch関数を使用してください。
        今日の日付: {datetime.today().strftime('%Y-%m-%d')}
        親切でプロフェッショナルに対応してください。""",
        hooks=[MemoryHookProvider(client, memory_id, ACTOR_ID, SESSION_ID)],
        tools=[websearch],
    )
    return agent

# エージェントを作成
agent = create_personal_agent()
logger.info("✅ メモリとウェブ検索機能付きパーソナルエージェントが作成されました")

#### おめでとうございます！エージェントの準備ができました！:) 
## エージェントをテストしましょう

In [ ]:
# メモリ付きで会話をテスト
print("=== 最初の会話 ===")
print(f"ユーザー: 私の名前はAlexで、AIについて学ぶことに興味があります。")
print(f"エージェント: ", end="")
agent("私の名前はAlexで、AIについて学ぶことに興味があります。")

In [ ]:
print(f"ユーザー: 2025年の最新のAIトレンドを検索してもらえますか？")
print(f"エージェント: ", end="")
agent("2025年の最新のAIトレンドを検索してもらえますか？")

In [ ]:
print(f"ユーザー: 特に機械学習の応用に興味があります。")
print(f"エージェント: ", end="")
agent("特に機械学習の応用に興味があります。")

## メモリ継続性のテスト

メモリシステムが正しく動作しているかテストするため、新しいエージェントインスタンスを作成して、以前に保存された情報にアクセスできるかを確認します：

In [ ]:
# 新しいエージェントインスタンスを作成（ユーザーが戻ってきた場合をシミュレート）
print("=== ユーザーが戻る - 新しいセッション ===")
new_agent = create_personal_agent()

# メモリ継続性をテスト
print(f"ユーザー: 私の名前は何でしたか？")
print(f"エージェント: ", end="")
new_agent("私の名前は何でしたか？")

print(f"ユーザー: 機械学習についてもっと情報を検索してもらえますか？")
print(f"エージェント: ", end="")
new_agent("機械学習についてもっと情報を検索してもらえますか？")

## 保存されたメモリの表示

In [ ]:
# メモリに保存されている内容を確認
print("=== メモリの内容 ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3 # kを調整してより多くまたは少ないターンを表示
)

for i, turn in enumerate(recent_turns, 1):
    print(f"ターン {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

## まとめ

このチュートリアルでは、パーソナルエージェントの構築方法を示しました。学習したこと：

- ストラテジーなしでメモリリソースを作成
- 会話履歴のため`get_last_k_turns`を使用
- エージェントにウェブ検索機能を追加
- コンテキスト読み込み用メモリフックの実装

**次のステップ：**
- より高度なツールを追加
- 長期記憶ストラテジーの実装
- 複数ソースでの検索機能の強化

## クリーンアップ（オプション）

In [ ]:
# メモリリソースを削除する場合はコメントアウトを解除
# client.delete_memory_and_wait(memory_id)
# logger.info(f"✅ メモリを削除しました: {memory_id}")